In [1]:
import sagemaker
from sklearn.model_selection import train_test_split
import boto3 #used to connect s3 bucket
import pandas as pd

sm_boto3 = boto3.client("sagemaker")
sess = sagemaker.Session()
region = sess.boto_session.region_name
bucket = 'sagemobbucket'
print("using bucket" + bucket)

using bucketsagemobbucket


In [3]:
df = pd.read_csv("train.csv")

In [4]:
df.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,...,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,...,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,...,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,...,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,...,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,...,1208,1212,1411,8,2,15,1,1,0,1


In [6]:
df.shape

(2000, 21)

In [7]:
df['price_range'].value_counts(normalize=True)

price_range
1    0.25
2    0.25
3    0.25
0    0.25
Name: proportion, dtype: float64

In [8]:
df.columns

Index(['battery_power', 'blue', 'clock_speed', 'dual_sim', 'fc', 'four_g',
       'int_memory', 'm_dep', 'mobile_wt', 'n_cores', 'pc', 'px_height',
       'px_width', 'ram', 'sc_h', 'sc_w', 'talk_time', 'three_g',
       'touch_screen', 'wifi', 'price_range'],
      dtype='object')

In [9]:
df.shape

(2000, 21)

In [10]:
df.isnull().mean() * 100
#finding null value

battery_power    0.0
blue             0.0
clock_speed      0.0
dual_sim         0.0
fc               0.0
four_g           0.0
int_memory       0.0
m_dep            0.0
mobile_wt        0.0
n_cores          0.0
pc               0.0
px_height        0.0
px_width         0.0
ram              0.0
sc_h             0.0
sc_w             0.0
talk_time        0.0
three_g          0.0
touch_screen     0.0
wifi             0.0
price_range      0.0
dtype: float64

In [11]:
features = list(df.columns)
features

['battery_power',
 'blue',
 'clock_speed',
 'dual_sim',
 'fc',
 'four_g',
 'int_memory',
 'm_dep',
 'mobile_wt',
 'n_cores',
 'pc',
 'px_height',
 'px_width',
 'ram',
 'sc_h',
 'sc_w',
 'talk_time',
 'three_g',
 'touch_screen',
 'wifi',
 'price_range']

In [12]:
label = features.pop(-1) #remove last feature
label

'price_range'

In [15]:
x = df[features]
y = df[label]

In [16]:
x.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0


In [17]:
y.head()

0    1
1    2
2    2
3    2
4    1
Name: price_range, dtype: int64

In [19]:
x.shape

(2000, 20)

In [22]:
y.value_counts()

price_range
1    500
2    500
3    500
0    500
Name: count, dtype: int64

In [24]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size=0.40, random_state=0)

In [25]:
print(xtrain.shape)
print(xtest.shape)
print(ytrain.shape)
print(ytest.shape)

(1200, 20)
(800, 20)
(1200,)
(800,)


In [27]:
trainX = pd.DataFrame(xtrain)
trainX[label] = ytrain

testX = pd.DataFrame(xtest)
testX[label] = ytest

In [28]:
print(trainX.shape)
print(testX.shape)

(1200, 21)
(800, 21)


In [29]:
trainX.to_csv("train-V-1.csv", index = False)
testX.to_csv("test-V-1.csv", index = False)

In [30]:
#send data to s3, sagemaker will train data from s3
sk_prefix = 'sagemaker/mobile_price_classification/sklearncontainer'
trainpath = sess.upload_data(
    path = "train-V-1.csv", bucket = bucket, key_prefix = sk_prefix
)

testpath = sess.upload_data(
    path = "test-V-1.csv", bucket = bucket, key_prefix = sk_prefix
)

In [31]:
print(trainpath)
print(testpath)

s3://sagemobbucket/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv
s3://sagemobbucket/sagemaker/mobile_price_classification/sklearncontainer/test-V-1.csv


upto this its an data ingestion phase. taken data done train test split, and uploaded into s3

In [34]:
%%writefile script.py

from sklearn.ensemble import RandomForestClassifer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
import sklearn
import joblib
import boto3
import pathlib
from io import StringIO
import argparse
import os
import numpy as np
import pandas as pd

def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf

if __name__=="__main__":
    print("[INFO] Extracting arguments")
    parser = argparse.ArgumentParser()

    parse.add_argument("--n_estimators", type = int, default=100)
    parse.add_argument("--random_state", type = int, default=0)

    #aws sagemaker arg
    parse.add_argument("--model-dir", type = str, default=os.environ.get("SM_MODEL_DIR"))
    parse.add_argument("--train", type = str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parse.add_argument("--test", type = str, default=os.environ.get("SM_CHANNEL_TEST"))
    parse.add_argument("--train-file", type = str, default="train-V-1.csv")
    parse.add_argument("--test-file", type = str, default="test-V-1.csv")

    args, _ = parser.parse_known_args()

    print("sklearn Version: ", sklearn.__version__)
    print("joblib Version: ", joblib.__version__)

    print("[INFO] Extracting arguments")
    print()
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    features = list(train_df.columns)
    label = features.pop(-1)

    print("Building training and testing dataset")
    print()
    xtrain = train_df[features]
    xtest = test_df[features]
    ytrain = train_df[label]
    ytest = test_df[label]

    print("column order: ")
    print(features)
    print()

    print("Label column is: ",label)
    print()

    print("Data shape: ")
    print()
    print("SHAPE OF TRAINING DATA")
    print(xtrain.shape)
    print(ytrain.shape)
    print()
    print("SHAPE OF TESTING DATA")
    print(xtest.shape)
    print(ytest.shape)
    print()

    print("traing RandomForest Model")
    print()
    model = RandomForestClassifier(n_estimatos = args.n_estimators, rando_state = args.random_state)
    model.fit(xtrain,ytrain)
    print()

    model_path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model,model_path)
    print("model persisted at " + model_path)
    print()

    y_pred_test = model.predict(xtest)
    test_acc = accuracy_score(ytest, y_pred_test)
    test_rep = classification_report(y_test, y_pred_test)

    print()
    print("metrics")
    print()
    print("total rows: ", xtest.shape[0])
    print("[testing] model accuracy is: ", test_acc)
    print("[testing] testing report: ")
    print(test_re)


Overwriting script.py


In [47]:
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point = "script.py",
    role = "arn:aws:iam::046531120064:role/service-role/AmazonSageMaker-ExecutionRole-20230815T222724",
    instance_count = 1,
    instance_type = "ml.m5.large",
    framework_version = FRAMEWORK_VERSION,
    base_job_name = "RF-custom-sklearn",
    hyperparamters = {
        "n_estimators:":100,
        "random_state": 0,
    },
    use_spot_instances = True,
    max_wait = 7200,
    max_run = 3600
)

In [48]:
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait = True)
#launching training job 

Using provided s3_resource


INFO:sagemaker:Creating training-job with name: RF-custom-sklearn-2023-08-15-16-59-07-861


ClientError: An error occurred (ValidationException) when calling the CreateTrainingJob operation: No S3 objects found under S3 URL "s3://sagemobbucket/sagemaker/mobile_price_classification/sklearncontainer/train-V-1.csv" given in input data source. Please ensure that the bucket exists in the selected region (ap-south-1), that objects exist under that S3 prefix, and that the role "arn:aws:iam::046531120064:role/service-role/AmazonSageMaker-ExecutionRole-20230815T222724" has "s3:ListBucket" permissions on bucket "sagemobbucket". Error message from S3: The bucket is in this region: us-east-1. Please use this region to retry the request

In [ ]:
#for deploying model 

from sagemaker.sklearn.model import SKLearnModel
from time import gmtime, strftime

model_name = "Custom-sklearn-model " + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model = SKLearnModel(
    name = model_name,
    model_data = artifact,
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

In [ ]:
model

In [ ]:
#for endpoint deploying model 
endpoint_name = "Custom-sklearn-model-" +strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName= {}".format(endpoint_name))

predictor = model.deploy(
    initial_instance_count=1,
    instance_type="ml.m4.xlarge",
    endpoint_name=endpoint_name,
)

In [ ]:
endpoint_name

In [ ]:
print(predictor,predict(testX[features][0.2].values.tolist()))

In [ ]:
sm_boto3.delete_endpoint(EndpointName= endpoint_name)